In [21]:
from google.cloud.sql.connector import Connector, IPTypes
import sqlalchemy
from google.oauth2 import service_account
from sqlalchemy import text
#pip install cloud-sql-python-connector

# 指定服务账户文件路径
SERVICE_ACCOUNT_FILE = './qf5214_lyj.json'
                                                            #各自修改成自己的密钥文件
# 创建服务账户凭据
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE
)

# 初始化Connector对象
connector = Connector(credentials=credentials)

def getconn() -> sqlalchemy.engine.base.Connection:
    # 连接信息
    instance_connection_string = "supple-folder-418707:us-central1:qf5214project"
    db_user = "user_wxc"
    db_pass = "5214group4_cloud_wxc"
    db_name = "ods"                                       # 替换为你的实际数据库名，不过没有好像也可以

    conn = connector.connect(
        instance_connection_string,
        "pymysql",  
        user=db_user,
        password=db_pass,
        db=db_name,
        ip_type=IPTypes.PUBLIC,  # 使用公共IP进行访问
    )
    return conn

# 创建SQLAlchemy engine
engine = sqlalchemy.create_engine(
    "mysql+pymysql://",  # 我们的数据库是MySQL8.0
    creator=getconn,
)

# 使用engine进行数据库操作
with engine.connect() as conn:
    result = conn.execute(text("SELECT NOW();"))  # 使用text()包裹原始SQL语句，否则报错
    for row in result:
        print(row)      #返回数据库时间


with engine.connect() as conn:
    # 执行查询以获取所有数据库的列表（MySQL版本）
    result = conn.execute(text("SHOW DATABASES;"))
    # 遍历结果并打印每个数据库名称
    for row in result:
        print(row[0])


(datetime.datetime(2024, 4, 14, 15, 30, 47),)
Bitcoin_data
ads
bitcoin_test1
bitcoin_wxc
dwd
dws
information_schema
mysql
ods
performance_schema
sys


In [2]:
conn = engine.connect()

# 执行查询以获取所有数据库的列表（MySQL版本）
conn.execute(text("use ods;"))
#conn.execute(text("drop table if exists news;"))

#conn.execute(text(create_table))
#conn.execute(text(load_csv))
#result = conn.execute(text("select * from news LIMIT 10;"))
result = conn.execute(text("show tables;"))
for row in result:
    print(row[0])
    

ods_market_btc_delta
ods_market_cpi_delta
ods_market_employment_delta
ods_market_eth_delta
ods_market_ffr_delta
ods_market_sofr_delta
ods_news_delta
ods_talk_comments_delta
ods_talk_subjects_delta
ods_talk_users_delta


In [29]:
command = conn.execute(text("DESCRIBE ods_market_employment_delta;"))
for row in command:
    print(row)

('date', 'date', 'NO', 'PRI', None, '')
('employment_nonfarm', 'int', 'YES', '', None, '')
('population', 'int', 'YES', '', None, '')
('employment_nonfarm_ratio', 'double', 'YES', '', None, '')


In [4]:
import requests
from datetime import datetime, timedelta

def fetch_bitcoin_price(date=None):
    if date is None:
        date = datetime.utcnow().strftime('%Y-%m-%d')  # 使用UTC时间

    # 设定当天的开始和结束时间戳
    start_of_day = datetime.strptime(date, "%Y-%m-%d")
    end_of_day = start_of_day + timedelta(days=1) - timedelta(seconds=1)  # 当天的23:59:59

    url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range'
    params = {
        'vs_currency': 'usd',
        'from': int(start_of_day.timestamp()),
        'to': int(end_of_day.timestamp())
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # 检查价格数据并返回当天最后一条记录的价格
    if 'prices' in data and data['prices']:
        return data['prices'][-1][1]  # 返回最后一条价格数据
    else:
        return None
        
def fetch_bitcoin_volume(date=None):
    if date is None:
        date = datetime.utcnow().strftime('%Y-%m-%d')  # 使用UTC时间

    # 设定当天的开始和结束时间戳
    start_of_day = datetime.strptime(date, "%Y-%m-%d")
    end_of_day = start_of_day + timedelta(days=1) - timedelta(seconds=1)  # 当天的23:59:59

    url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range'
    params = {
        'vs_currency': 'usd',
        'from': int(start_of_day.timestamp()),
        'to': int(end_of_day.timestamp())
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # 检查交易量数据并返回当天最后一条记录的交易量
    if 'total_volumes' in data and data['total_volumes']:
        return data['total_volumes'][-1][1]  # 返回最后一条交易量数据
    else:
        return None
        
price = fetch_bitcoin_price()
volume = fetch_bitcoin_volume()

current_date = datetime.utcnow().strftime('%Y-%m-%d')
conn.execute(
    text("REPLACE INTO ods_market_btc_delta (date, btc_price, btc_volume) VALUES (:date, :btc_price, :btc_volume);"),
    {'date': current_date, 'btc_price': price, 'btc_volume': volume}
)
result = conn.execute(text("SELECT * FROM ods_market_btc_delta ORDER BY date DESC;"))
for row in result:
    print(row)
#fetch_bitcoin_price()


(datetime.date(2024, 4, 14), Decimal('64211.1661'), Decimal('61507240157.9903'))


In [34]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def cpi_function(date=None, api_key='7de51df1d77defbeae082211557f4d6a'):
    if date is None:
        current_date = pd.Timestamp.today()

    cpi_series_id = 'CPIAUCNS'  # CPI All Urban Consumers
    
    while True:
        date = current_date.strftime('%Y-%m-%d')
        
        cpi_api_url = f'https://api.stlouisfed.org/fred/series/observations?series_id={cpi_series_id}&api_key={api_key}&file_type=json&observation_start={date}&observation_end={date}'
        cpi_response = requests.get(cpi_api_url)
        cpi_response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        cpi_data = cpi_response.json()
    
        # Check if observations are available
        if 'observations' not in cpi_data or not cpi_data['observations']:
            current_date -= timedelta(days = 1)  # No data available for the specified date, move to the previous day
            continue
        else:
            cpi_value = cpi_data['observations'][0]['value']
        return float(cpi_value), date

cpi_update, valid_date= cpi_function()
current_date = pd.Timestamp.today().strftime('%Y-%m-%d')
if valid_date == current_date:
    conn.execute(
        text("REPLACE INTO ods_market_cpi_delta (date, cpi) VALUES (:date, :cpi);"),
        {'date': valid_date, 'cpi': cpi_update}
    )
else:
    conn.execute(
        text("REPLACE INTO ods_market_cpi_delta (date, cpi) VALUES (:date, :cpi);"),
        {'date': current_date, 'cpi': None}
    )
result = conn.execute(text("SELECT * FROM ods_market_cpi_delta ORDER BY date DESC;"))
for row in result:
    print(row)

(datetime.date(2024, 4, 14), None)


In [33]:
import requests
import pandas as pd
from datetime import timedelta

def non_farmer_function(date=None, api_key='7de51df1d77defbeae082211557f4d6a'):
    if date is None:
        current_date = pd.Timestamp.today()

    population_series_id = 'POPTHM'  # Total Population
    payems_series_id = 'PAYEMS'  # Nonfarm Payroll Employment

    # Fetch population data
    while True:
        date = current_date.strftime('%Y-%m-%d')
        
        population_api_url = f'https://api.stlouisfed.org/fred/series/observations?series_id={population_series_id}&api_key={api_key}&file_type=json&observation_start={date}&observation_end={date}'
        population_response = requests.get(population_api_url)
        population_data = population_response.json()
    
        # Check if observations are available
        if 'observations' not in population_data or not population_data['observations']:
            current_date -= timedelta(days = 1)  # No data available for the specified date, move to the previous day
            continue
        else:
            total_population = float(population_data['observations'][0]['value'])
    
        # Fetch nonfarm employment data
        payems_api_url = f'https://api.stlouisfed.org/fred/series/observations?series_id={payems_series_id}&api_key={api_key}&file_type=json&observation_start={date}&observation_end={date}'
        payems_response = requests.get(payems_api_url)
        payems_data = payems_response.json()
        
        # Check if observations are available
        if 'observations' not in payems_data or not payems_data['observations']:
            current_date -= timedelta(days = 1)  # No data available for the specified date, move to the previous day
            continue        
        else:
            nonfarm_employment = float(payems_data['observations'][0]['value'])

        # Calculate nonfarm population ratio
        nonfarm_population_ratio = nonfarm_employment / total_population
        return nonfarm_employment, total_population, nonfarm_population_ratio, date
    
data = non_farmer_function()

current_date = pd.Timestamp.today().strftime('%Y-%m-%d')
if current_date == data[3]:
    conn.execute(
        text("REPLACE INTO ods_market_employment_delta (date, employment_nonfarm, population, employment_nonfarm_ratio) VALUES (:date, :employment_nonfarm, :population, :employment_nonfarm_ratio);"),
        {'date': data[3], 'employment_nonfarm': data[0], 'population': data[1], 'employment_nonfarm_ratio': data[2]}
    )
else:
    conn.execute(
    text("REPLACE INTO ods_market_employment_delta (date, employment_nonfarm, population, employment_nonfarm_ratio) VALUES (:date, :employment_nonfarm, :population, :employment_nonfarm_ratio);"),
        {'date': current_date, 'employment_nonfarm': None, 'population': None, 'employment_nonfarm_ratio': None}
    )

result = conn.execute(text("SELECT * FROM ods_market_employment_delta ORDER BY date DESC;"))
for row in result:
    print(row)


(datetime.date(2024, 4, 14), None, None, None)


In [61]:
import requests
from datetime import datetime

def get_fedfunds_data(date='', api_key='051f69ecb49f2cecf43a9ad162820cd1'):
    # 检查日期输入，如果为空则使用当前日期
    if not date:
        date = datetime.now().strftime("%Y-%m-%d")

    # 构建请求URL
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": "FEDFUNDS",
        "api_key": api_key,
        "file_type": "json",
        "observation_start": date,
        "observation_end": date,
    }

    # 发送GET请求
    response = requests.get(url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        # 确保观测数据存在且不为空
        if 'observations' in json_data and json_data['observations']:
            # 提取并返回利率值
            value = json_data['observations'][0]['value']
            return value
        else:
            return None
    else:
        print("")
        return None
rate = get_fedfunds_data()
current_date = datetime.utcnow().strftime('%Y-%m-%d')
conn.execute(
    text("REPLACE INTO ods_market_ffr_delta (date, ffr) VALUES (:date, :ffr);"),
    {'date': current_date, 'ffr': rate}
)
result = conn.execute(text("SELECT * FROM ods_market_btc_delta ORDER BY date DESC;"))
for row in result:
    print(row)

(datetime.date(2024, 4, 14), Decimal('64434.0620'), Decimal('55672903271.2681'))


In [16]:
#pip install gdeltdoc

from gdeltdoc import GdeltDoc, Filters, repeat
import pandas as pd
from datetime import datetime, timedelta
from textblob import TextBlob

result_df = pd.DataFrame()


start_date = datetime.today().date() - timedelta(days=1)
end_date = datetime.today().date() - timedelta(days=1)
error = list()

current_date = start_date
while current_date <= end_date:

    next_date = current_date + timedelta(days=1)
    
    current_date_str = current_date.strftime('%Y-%m-%d')
    next_date_str = next_date.strftime('%Y-%m-%d')
    
    f = Filters(
        keyword = "bitcoin",
        start_date = current_date_str,
        end_date =  next_date_str,
        num_records = 250,
        
        
        #domain = ["bbc.co.uk", "nytimes.com", "yahoo.com", "newsbtc.com", "coindesk.com",
        #          "twitter.com", "medium.com", "bitcoinmagazine.com", "reddit.com"],
        
        
        country = ["US", "UK", "CN", "JA", "GM"],
        repeat = repeat(5, "bitcoin")
    )

    gd = GdeltDoc()
    try:
        articles = gd.article_search(f)
    except Exception as e:
        print('导出异常，跳过！！')
        error.append(current_date)
        current_date = next_date
        continue
    #timeline = gd.timeline_search("timelinevol", f)
    
    result_df = pd.concat([result_df, articles], ignore_index=True)
    
    # 将日期向前推进一天
    current_date = next_date
    
result_df = result_df.drop_duplicates()

#bitcoin_20170101_20170630_big = r'C:\Users\User\Desktop\2024_2.xlsx'
#result_df.to_excel(bitcoin_20170101_20170630_big, sheet_name="Sheet1", index=False)
columns_to_keep = ['url', 'title', 'domain', 'language', 'sourcecountry']
valid_df = result_df[columns_to_keep]
new_column_names = {
    'url': 'news_url',
    'title': 'news_title',
    'domain': 'news_domain',
    'language': 'news_language',
    'sourcecountry': 'news_country'
}

# 重命名列
df = valid_df.rename(columns=new_column_names)

def sentiment_polarity(text):
    if pd.isna(text):
        return 0.0
    return TextBlob(text).sentiment.polarity

def sentiment_subjectivity(text):
    if pd.isna(text):
        return 0.5
    return TextBlob(text).sentiment.subjectivity

# Apply the sentiment analysis functions to the 'title' column
df['news_title_emotion'] = df['news_title'].apply(sentiment_polarity)
df['news_title_subjectivity'] = df['news_title'].apply(sentiment_subjectivity)


result = conn.execute(text("SELECT COUNT(news_id) FROM dwd.dwd_news_df;"))
for i in result:
    news_number = i[0]
    
df = df.reset_index()
df['news_id'] = df['index'] + news_number + 1
df.drop('index', axis=1, inplace=True)
current_date = datetime.now().strftime('%Y-%m-%d')
df['date'] = current_date

correct_column_order = [
    'news_id', 'date', 'news_title', 'news_url', 'news_domain', 
    'news_language', 'news_country', 'news_title_emotion', 'news_title_subjectivity'
]

df = df[correct_column_order]

for index, row in df.iterrows():
    command = f"""REPLACE INTO ods_news_delta (news_id, date, news_title, news_url, news_domain, news_language, news_country, news_title_emotion, news_title_subjectivity) 
    VALUES ('{row['news_id']}', '{row['date']}', '{row['news_title']}', '{row['news_url']}', '{row['news_domain']}', 
    '{row['news_language']}', '{row['news_country']}', '{row['news_title_emotion']}','{row['news_title_subjectivity']}');"""
    conn.execute(text(command))
result = conn.execute(text("SELECT * FROM ods_news_delta LIMIT 10;"))
for row in result:
    print(row)

(190328, datetime.date(2024, 4, 14), 'Hong Kong Approval of Bitcoin ETFs Could Ignite Pre - Halving Price Rally', 'http://www.econotimes.com/Hong-Kongs-Approval-of-Bitcoin-ETFs-Could-Ignite-Pre-Halving-Price-Rally-1675549', 'econotimes.com', 'English', 'United States', 0.0, 0.0)
(190329, datetime.date(2024, 4, 14), 'Bitcoin Cash ( BCH ) Achieves Market Capitalization of $10 . 43 Billion', 'https://www.wkrb13.com/2024/04/13/bitcoin-cash-bch-achieves-market-capitalization-of-10-43-billion.html', 'wkrb13.com', 'English', 'United States', 0.0, 0.0)
(190330, datetime.date(2024, 4, 14), 'Bitcoin Cash ( BCH ) Achieves Market Capitalization of $10 . 43 Billion', 'https://www.dailypolitical.com/2024/04/13/bitcoin-cash-bch-achieves-market-capitalization-of-10-43-billion.html', 'dailypolitical.com', 'English', 'United States', 0.0, 0.0)
(190331, datetime.date(2024, 4, 14), 'Get ready , the Bitcoin  halving  is coming', 'https://nypost.com/2024/04/13/business/get-ready-the-bitcoin-halving-is-comin

In [17]:
import requests
from datetime import datetime
import pandas as pd

def get_sofr_data(date=None, api_key='051f69ecb49f2cecf43a9ad162820cd1'):
    
    date = pd.Timestamp.today().strftime("%Y-%m-%d")
    # 构建请求URL
    url = f"https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": "FEDFUNDS",
        "api_key": api_key,
        "file_type": "json",
        "observation_start": date,
        "observation_end": date,
    }

    # 发送GET请求
    response = requests.get(url, params=params)

    if response.status_code == 200:
        json_data = response.json()
        if 'observations' in json_data:
            # 获取数据
            try:
                value = json_data['observations'][0]['value']
            except Exception as e:
                date = pd.to_datetime(date) - timedelta(days=30)
                date = str(date)[0:10]
                url = f"https://api.stlouisfed.org/fred/series/observations"
                params = {
                    "series_id": "FEDFUNDS",
                    "api_key": api_key,
                    "file_type": "json",
                    "observation_start": date,
                    "observation_end": date,
                }
                response = requests.get(url, params=params)
                json_data = response.json()
                if 'observations' in json_data:
                    value = json_data['observations'][0]['value']
                else:
                    return None
            return value
        else:
            return None
    else:
        print(f"Failed to retrieve interest rate data for {date}, status code: {response.status_code}")
        return None
rate = get_sofr_data()
current_date = pd.Timestamp.today().strftime("%Y-%m-%d")
conn.execute(
    text("REPLACE INTO ods_market_sofr_delta (date, sofr) VALUES (:date, :sofr);"),
    {'date': current_date, 'sofr': rate}
)
result = conn.execute(text("SELECT * FROM ods_market_sofr_delta ORDER BY date DESC;"))
for row in result:
    print(row)


(datetime.date(2024, 4, 14), Decimal('5.3300'))


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from datetime import date,timedelta,datetime
import re
import os

def getComment(driver,formatted_date):
    pagenow = 1
    comment = []
    flag = True
    users = []
    dates = []
    acts = []
    merit = []
    while flag == True:
        dt = driver.find_elements(By.XPATH,"//tbody//tr//td[@valign='middle']//div[@class='smalltext']")
        content = driver.find_elements(By.XPATH,"//td[@class='td_headerandpost']//div[@class='post']")
        user = driver.find_elements(By.XPATH,"//td[@class='poster_info']//b//a")
        act = driver.find_elements(By.XPATH,"//td[@class='poster_info']//div[@class='smalltext']")
        content_list = []
        dt_list = []
        user_list = []
        act_list = []
        for i in range(min(len(content),len(user),len(act))):
            content_list.append(content[i].get_attribute('textContent'))
            user_list.append(user[i].get_attribute('textContent'))
            act_list.append(act[i].get_attribute('textContent'))
        for j in range(len(dt)):
            dt_list.append(dt[j].get_attribute('textContent'))
        indexes_to_remove2 = [index for index, item in enumerate(dt_list) if item.startswith('Merited')]
        for index2 in sorted(indexes_to_remove2, reverse=True):
            del dt_list[index2]
        indexes_to_remove = [index for index, item in enumerate(dt_list) if item[0].isdigit()]
        for index in sorted(indexes_to_remove, reverse=True):
            del content_list[index]
            del dt_list[index]
            del act_list[index]
            del user_list[index]
        
        for z in range(min(len(dt_list),len(user_list),len(content_list),len(act_list))):
            comment.append(content_list[z])
            users.append(user_list[z])
            dates.append(dt_list[z])
            match1 = re.search(r'Activity: (\d+)', act_list[z])
            activity_number = match1.group(1) if match1 else None
            acts.append(activity_number)
            match2 = re.search(r'Merit: (\d+)', act_list[z])
            merit_number = match2.group(1) if match2 else None
            merit.append(merit_number)
        page = driver.find_elements(By.XPATH,"//a[@class='navPages']")
        for k in range(len(page)):
            try:
                value = int(page[k].get_attribute('textContent'))
            except:
                value = 0
            if value == pagenow + 1:
                pagenow = pagenow+1
                page[k].click()
                time.sleep(2)
                break
        else:
            flag = False
    dates_to_remain = [index for index, item in enumerate(dates) if item.startswith('T')]
    comment = [comment[a] for a in dates_to_remain if a<len(comment)]
    dates = [dates[b] for b in dates_to_remain if b<len(dates)]
    users = [users[c] for c in dates_to_remain if c<len(users)]
    acts = [acts[d] for d in dates_to_remain if d<len(acts)]
    merit = [merit[e] for e in dates_to_remain if e<len(merit)]
    for w in range(len(dates)):
        if dates[w].startswith('T'):
            dates[w]  = formatted_date
    return comment,dates,users,acts,merit
def commentToday():
    path = r'./chromedriver.exe'
    cService = webdriver.ChromeService(executable_path=r'./chromedriver.exe')
    opt = Options()
    opt.add_argument("--headless") 
    driver = webdriver.Chrome(service = cService,options=opt)
    url = 'https://bitcointalk.org/index.php?board=1.0'
    driver.get(url)
    current_date = datetime.now()
    formatted_date = current_date.strftime('%B %d, %Y')
    df = pd.DataFrame(columns=['subject','replies','views','comment','date','user','activity','merit'])
    a = driver.find_elements(By.XPATH,"//td[@class='windowbg']//span/a")
    b = driver.find_elements(By.XPATH, "//td[@class='windowbg' and @valign='middle' and @width='4%' and @align='center']")
    c = driver.find_elements(By.XPATH, "//td[contains(@class, 'windowbg2') and contains(@class, 'lastpostcol')]//span[@class='smalltext']")
    url_list = []
    sub_list = []
    rep_list = []
    view_list = []
    date_list = []
    for j in range(min(len(a),len(b)//2)):
        url_list.append(a[j].get_attribute('href'))
        sub_list.append(a[j].get_attribute('textContent'))
        v1 = b[2*j].get_attribute('textContent').replace('\n', '').replace('\t', '')
        v2 = b[2*j+1].get_attribute('textContent').replace('\n', '').replace('\t', '')
        d1 = c[j].get_attribute('textContent').replace('\n', '').replace('\t', '')
        rep_list.append(v1)
        view_list.append(v2)
        date_list.append(d1)

    post_to_view = [index for index, item in enumerate(date_list) if item.startswith('T')]
    url_list = [url_list[k] for k in post_to_view if k<len(url_list)]

    for z in range(len(url_list)):
        current_url = driver.current_url
        driver.get(url_list[z])
        c,d,u,a,m = getComment(driver,formatted_date)
        for i in range(len(c)):
            ct = [sub_list[z],rep_list[z],view_list[z],c[i],d[i],u[i],a[i],m[i]]
            ctm = pd.DataFrame(ct).T
            ctm.columns=['subject','replies','views','comment','date','user','activity','merit']
            df = pd.concat([df,ctm])

        df = df.reset_index(drop=True)
        time.sleep(3)
        driver.get(current_url) 
    df_subject = df.drop_duplicates(subset=['subject'])[['subject','replies', 'views']]
    df_comment = df[['subject','comment','date','user']]
    df_user = df.drop_duplicates(subset=['user'])[['user','activity', 'merit']]
    return df_subject,df_comment,df_user
df_subject,df_comment,df_user = commentToday()

In [22]:
'''ods_talk_comments_delta
ods_talk_subjects_delta
ods_talk_users_delta'''
command = conn.execute(text("describe ods_talk_subjects_delta;"))
for row in command:
    print(row)


('talk_subject_id', 'varchar(300)', 'NO', 'PRI', None, '')
('date', 'date', 'YES', '', None, '')
('talk_subject_replies', 'int', 'YES', '', None, '')
('talk_subject_views', 'int', 'YES', '', None, '')
('talk_subject_emotion', 'double', 'YES', '', None, '')
('talk_subject_subjectivity', 'double', 'YES', '', None, '')


In [23]:
command = conn.execute(text("describe ods_talk_comments_delta;"))
for row in command:
    print(row)


('talk_comment_id', 'int', 'NO', 'PRI', None, '')
('date', 'date', 'YES', '', None, '')
('talk_subject_id', 'varchar(300)', 'YES', '', None, '')
('talk_user_id', 'varchar(100)', 'YES', '', None, '')
('talk_comment', 'text', 'YES', '', None, '')
('talk_comment_emotion', 'double', 'YES', '', None, '')


In [24]:
command = conn.execute(text("describe ods_talk_users_delta;"))
for row in command:
    print(row)


('talk_user_id', 'varchar(100)', 'NO', 'PRI', None, '')
('talk_user_activity', 'int', 'YES', '', None, '')
('talk_user_merit', 'int', 'YES', '', None, '')


In [12]:
df_subject

,talk_subject_id,talk_subject_replies,talk_subject_views,date
0,Bitcoin puzzle transaction ~32 BTC prize to wh...,4890,182347,2024-04-14
5,Gold vs bitcoin in a WWIII scenario,4,17,2024-04-14
13,Short term Investors feeling cool with the cen...,8,43,2024-04-14
14,bullish & bearish community !!!!,4,23,2024-04-14
15,Bitcoin = Proof of Scam ?,39,290,2024-04-14
16,[READ] Bitcoin history from 2008 to 2016!,41,544,2024-04-14
35,Know your facts: Fundamental differences betwe...,24,212,2024-04-14
48,How do you know which companies to trust? Help...,0,6,2024-04-14
50,It takes the balls...,17,92,2024-04-14
81,Bitcoin halving countdown,39,264,2024-04-14


In [ ]:
import pandas as pd
from textblob import TextBlob
import re
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

def sentiment_polarity(text):
    if pd.isna(text):
        return 0.0
    return TextBlob(text).sentiment.polarity

def sentiment_subjectivity(text):
    if pd.isna(text):
        return 0.5
    return TextBlob(text).sentiment.subjectivity
    
new_column_names = {
    'subject': 'talk_subject_id',
    'replies': 'talk_subject_replies',
    'views': 'talk_subject_views',
}

# 重命名列
df_subject = df_subject.rename(columns=new_column_names)
current_date = datetime.now().strftime('%Y-%m-%d')
df_subject['date'] = current_date
df_subject['talk_subject_emotion'] = df_subject['talk_subject_id'].apply(sentiment_polarity)
df_subject['talk_subject_subjectivity'] = df_subject['talk_subject_id'].apply(sentiment_subjectivity)

# Replacement mappings
replacements = {
    r'\bthats\b': 'that is',
    r'\bive\b': 'i have',
    r'\bim\b': 'i am',
    r'\bya\b': 'yeah',
    r'\bcant\b': 'can not',
    r'\bwont\b': 'will not',
    r'\bid\b': 'i would',
    r'wtf': 'what the fuck',
    r'\bwth\b': 'what the hell',
    r'\br\b': 'are',
    r'\bu\b': 'you',
    r'\bk\b': 'ok',
    r'\bsux\b': 'sucks',
    r'\bno+\b': 'no',
    r'\bcoo+\b': 'cool',
    r'\bath\b': 'all time high',
    r'\batl\b': 'all time low',
    r'\bbtfd\b': 'buy the fucking dip',
    r'\bico\b': 'initial coin offering',
    r'\bfomo\b': 'fear of missing out',
    r'\bfud\b': 'fear uncertainty doubt',
    r'\bfucking\b': 'fuck',
    r'\bfudster\b': 'fear uncertainty doubt spreader',
    r'\broi\b': 'return on investment',
    r'\bmacd\b': 'moving average convergence divergence',
    r'\bpoa\b': 'proof of authority',
    r'\bpow\b': 'proof of work'
}

# Function to replace text according to the replacement dictionary
def replace_lingo(text):
    for pattern, replacement in replacements.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

# Function to remove URLs
def remove_urls(text):
    return re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|stratum[+]tcp?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

# Function to strip non-ASCII characters
def strip_non_ascii(string):
    return ''.join(c for c in string if 0 < ord(c) < 127)

# Applying the cleaning functions
df_comment['clean_comment'] = df_comment['comment'].str.lower()
df_comment['clean_comment'] = df_comment['clean_comment'].apply(remove_urls)
df_comment['clean_comment'] = df_comment['clean_comment'].apply(replace_lingo)
df_comment['clean_comment'] = df_comment['clean_comment'].apply(strip_non_ascii)

# Initialize NLTK's sentiment analyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

# Define a function for sentiment analysis
def sentiment_analysis(text):
    return analyzer.polarity_scores(text)['compound']

# Apply sentiment analysis

df_comment['talk_comment_emotion'] = df_comment['clean_comment'].apply(sentiment_analysis)
df_comment.drop('clean_comment', axis=1, inplace=True)
new_column_names = {
    'subject': 'talk_subject_id',
    'comment': 'talk_comment',
    'user': 'talk_user_id',
}
df_comment['date'] = current_date
df_comment = df_comment.reset_index()
df_comment['talk_comment_id'] = df_comment['index']
df_comment.drop('index', axis=1, inplace=True)
df_comment = df_comment.rename(columns=new_column_names)

new_column_names = {
    'user': 'talk_user_id',
    'activity': 'talk_user_activity',
    'merit': 'talk_user_merit',
}
df_user = df_user.rename(columns=new_column_names)


In [29]:
df_subject.to_sql('ods_talk_subjects_delta',engine,chunksize=10000,if_exists="replace",index=False)
df_comment.to_sql('ods_talk_comments_delta',engine,chunksize=10000,if_exists="replace",index=False)
df_user.to_sql('ods_talk_users_delta',engine,chunksize=10000,if_exists="replace",index=False)

106